In [1]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import warnings
import asyncio
import time
from datetime import datetime
pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

if not mt5.initialize():
    print("initialize() failed")
else:
    print("MT5 successfully initialised.\n")

MT5 successfully initialised.



In [2]:
# symbols =['AXREIT', 'TIMECOM', 'ABMB', 'MASTER', 'HARBOUR', 
#           'DIALOG', 'UOADEV', 'KAWAN', 'INARI', 'GASMSIA', 
#           'AHEALTH', 'HSPLANT', 'IJM']

# price_df = mt2.download_stocks(symbols)
########################
# def extract_number(col_name):
#     return int(''.join(filter(str.isdigit, col_name)))
# rename_sym = ['5106.KL','5031.KL','2488.KL','7029.KL','2062.KL',
#               '7277.KL','5249.KL','7216.KL','0166.KL','4677.KL',
#               '7090.KL','5138.KL','3336.KL']

# price_df.columns = rename_sym
# #Sort DataFrame columns using the custom key
# sorted_columns = sorted(price_df.columns, key=extract_number)
# price_df = price_df[sorted_columns]
#########################
# price_df.plot(figsize=(8,4))
# plt.xlabel('Year')
# plt.ylabel('Price')
# plt.show()

In [13]:
symbols = ['0166.KL', '2062.KL', '2488.KL', '3069.KL', '3336.KL', 
        '5031.KL', '5106.KL', '5138.KL', '5200.KL', '7029.KL', 
        '7090.KL', '7216.KL', '7293.KL']
symbols = [symbol + "SE" for symbol in symbols]
api_token = "x.x"
data = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(data)
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)

In [15]:
weights_df.tail(3)

,0166.KLSE,2062.KLSE,2488.KLSE,3069.KLSE,3336.KLSE,5031.KLSE,5106.KLSE,5138.KLSE,5200.KLSE,7029.KLSE,7090.KLSE,7216.KLSE,7293.KLSE
time,,,,,,,,,,,,,
2023-11-07,0.0,0.0,0.046063,0.0,0.249468,0.267288,0.139223,0.0,0.0,0.0,0.0,0.16898,0.128977
2023-11-08,0.0,0.0,-0.000000,0.0,0.000000,0.308983,0.177789,-0.0,0.0,0.0,0.0,0.29071,0.222518
2023-11-09,0.0,0.0,0.171601,0.0,0.000000,0.296659,0.371860,-0.0,0.0,0.0,0.0,-0.00000,0.159880


In [5]:
# ###dummy weights to test
# import random
# def generate_random_series(tickers, seed=None):
#     if seed is not None:
#         random.seed(seed)
#     random_values = [random.random() for _ in tickers]
#     total = sum(random_values)
#     normalized_values = [value / total for value in random_values]
#     return pd.Series(normalized_values, index=tickers)
# latest_weights = generate_random_series(symbols)


In [6]:
total_investment = 100000 #TRY 15K 10K 5K
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')
if previous_allocation_df is not None and previous_portfolio_df is not None:
    # Process the existing CSV data
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    # Handle the initial run scenario
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1
)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1
)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)

for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]

########################---async start here---#########################
print(allocation_df)
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(5)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')
########################---end here----#################################
for order in deleted_volumes:
    symbol = order['symbol']
    volume_filled = order['volume_filled']
    if order['direction'] == mt5.ORDER_TYPE_BUY_LIMIT:
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Entry Units'] = volume_filled
    else: 
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Exit Units'] = volume_filled

    #allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Allocated Units'] = volume_filled
# async def execute_and_monitor(allocation_df, timeout=60):
#     print(allocation_df)
#     order_ids = mt2.execute_trades_from_data(allocation_df)
#     print(f'Waiting for {timeout/60} minutes..')
#     await asyncio.sleep(timeout)
#     orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]  # Renamed from orders_list
#     print(f'orders_list: {orders_list}')
#     resubmitted_orders = mt2.delete_and_resubmit_orders(orders_list)  # Renamed from resubmitted_orders
#     print(f'resubmitted_orders: {resubmitted_orders}')
#     return orders_list, resubmitted_orders  # Return the renamed variables

# # Execute the async function and capture the returned values with the new variable names
# orders_list, resubmitted_orders = asyncio.run(execute_and_monitor(allocation_df, timeout=90))

# Calculate the portfolio value
entry_unit = allocation_df['Entry Units']
exit_unit = allocation_df['Exit Units']
share_price = allocation_df['Share Price']
allocated_unit = allocation_df['Allocated Units']
if previous_allocation_df is not None and previous_portfolio_df is not None: #2nd run onwards
    starting_cash = previous_portfolio_df['Cash Value'].iloc[-1]
    current_share_value = ((entry_unit*share_price)-(exit_unit*share_price)).sum()
    current_share_value += previous_portfolio_df['Portfolio Value'].iloc[-1]

else: #first run
    starting_cash = total_investment
    current_share_value = (entry_unit*share_price).sum()

portfolio_df = mt2.compile_portfolio_data(allocation_df, starting_cash, current_share_value)

mt2.save_df_to_csv(portfolio_df, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)
mt2.save_df_to_csv(allocation_df, 
                   folder_name='daily_allocation', 
                   file_name='allocation')


Folder not found: daily_allocation
Folder not found: daily_portfolio
This is the initial run. No previous allocation data available.
   Share Symbol  Share Price  Allocated Units   Weights  Holding Units  \
0     5106.KLSE         1.77          21000.0  0.371860            0.0   
1     5031.KLSE         5.18           5700.0  0.296659            0.0   
2     2488.KLSE         3.38           5000.0  0.171601            0.0   
3     7029.KLSE         2.88              0.0  0.000000            0.0   
4     2062.KLSE         1.18              0.0  0.000000            0.0   
5     7293.KLSE         2.42           6600.0  0.159880            0.0   
6     5200.KLSE         1.72              0.0  0.000000            0.0   
7     7216.KLSE         1.85              0.0 -0.000000            0.0   
8     0166.KLSE         2.96              0.0  0.000000            0.0   
9     3069.KLSE         3.54              0.0  0.000000            0.0   
10    7090.KLSE         2.48              0.0  0.0000

In [7]:
deleted_volumes

[]

In [8]:
print(f'starting_cash: {starting_cash}')
print(f'current_share_value: {current_share_value}')

starting_cash: 100000
current_share_value: 99568.0


In [9]:
portfolio_df

,Starting Capital,Purchase Cost,Sale Proceed,Cash Value,Portfolio Value,Total Value
0,100000,99568.0,0.0,432.0,99568.0,100000.0


In [10]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,5106.KLSE,1.77,21000.0,0.371860,0.0,21000.0,0.0
1,5031.KLSE,5.18,5700.0,0.296659,0.0,5700.0,0.0
2,2488.KLSE,3.38,5000.0,0.171601,0.0,5000.0,0.0
3,7029.KLSE,2.88,0.0,0.000000,0.0,0.0,0.0
4,2062.KLSE,1.18,0.0,0.000000,0.0,0.0,0.0
5,7293.KLSE,2.42,6600.0,0.159880,0.0,6600.0,0.0
6,5200.KLSE,1.72,0.0,0.000000,0.0,0.0,0.0
7,7216.KLSE,1.85,0.0,-0.000000,0.0,0.0,0.0
8,0166.KLSE,2.96,0.0,0.000000,0.0,0.0,0.0
9,3069.KLSE,3.54,0.0,0.000000,0.0,0.0,0.0


In [11]:
price_df

,5106.KLSE,5031.KLSE,2488.KLSE,7029.KLSE,2062.KLSE,7293.KLSE,5200.KLSE,7216.KLSE,0166.KLSE,3069.KLSE,7090.KLSE,5138.KLSE,3336.KLSE
time,,,,,,,,,,,,,
2023-10-23,1.7944,5.23,3.35,2.80,1.14,2.39,1.68,1.84,2.84,3.37,2.42,1.84,1.82
2023-10-24,1.8044,5.25,3.35,2.80,1.16,2.40,1.69,1.84,2.92,3.38,2.41,1.84,1.86
2023-10-25,1.7944,5.29,3.35,2.79,1.16,2.46,1.70,1.87,2.94,3.38,2.41,1.85,1.86
2023-10-26,1.7845,5.25,3.36,2.78,1.16,2.40,1.70,1.87,2.87,3.38,2.42,1.84,1.90
2023-10-27,1.7944,5.22,3.37,2.82,1.17,2.42,1.70,1.87,2.82,3.42,2.43,1.83,1.90
2023-10-30,1.7944,5.22,3.38,2.77,1.17,2.40,1.68,1.87,2.83,3.42,2.42,1.83,1.89
2023-10-31,1.8044,5.32,3.43,2.77,1.16,2.45,1.69,1.87,2.85,3.46,2.41,1.79,1.90
2023-11-01,1.7944,5.25,3.40,2.80,1.16,2.43,1.69,1.86,2.82,3.45,2.44,1.80,1.87
2023-11-02,1.8044,5.26,3.41,2.83,1.17,2.44,1.71,1.85,2.89,3.51,2.42,1.80,1.88
